In [1]:
!pip install --upgrade keras-nlp
!pip install --upgrade keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.2/572.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 4.7 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: keras-nlp
    Found existing installation: keras-nlp 0.14.0
    Uninstalling keras-nlp-0.14.0:
      Successfully uninstalled keras-nlp-0.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.7 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not current

In [2]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"  

import keras_nlp
import tensorflow as tf 
import tensorflow_datasets as tfds

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

2024-08-13 13:42:17.483366: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-13 13:42:17.483476: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 13:42:17.608784: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
start_char = 1
oov_char = 2
index_from = 3

In [4]:
(x_train, y_train),  (x_test, y_test) = tf.keras.datasets.reuters.load_data(start_char=start_char, oov_char=oov_char, index_from=index_from)

2110848/2110848 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
X = np.concatenate((x_train, x_test), axis=0)
Y = np.concatenate((y_train, y_test), axis=0)

In [6]:
X.shape

(11228,)

In [7]:
Y.shape

(11228,)

In [8]:
df = pd.concat([pd.DataFrame({'X': X, 'Y': Y})], ignore_index=True)
df.head()

,X,Y
0,"[1, 27595, 28842, 8, 43, 10, 447, 5, 25, 207, ...",3
1,"[1, 3267, 699, 3434, 2295, 56, 16784, 7511, 9,...",4
2,"[1, 53, 12, 284, 15, 14, 272, 26, 53, 959, 32,...",3
3,"[1, 4, 686, 867, 558, 4, 37, 38, 309, 2276, 46...",4
4,"[1, 8295, 111, 8, 25, 166, 40, 638, 10, 436, 2...",4


In [9]:
word_index = tf.keras.datasets.reuters.get_word_index()

inverted_word_index = dict(
    (i + index_from, word) for (word, i) in word_index.items()
)

inverted_word_index[start_char] = ""
inverted_word_index[oov_char] = ""

550378/550378 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [10]:
encode = lambda text : [word_index[word] for word in text.split()]
decode = lambda inp : " ".join([inverted_word_index[idx] for idx in inp])

In [11]:
df["text"] = df["X"].apply(decode)

In [12]:
df.head()

,X,Y,text
0,"[1, 27595, 28842, 8, 43, 10, 447, 5, 25, 207, ...",3,mcgrath rentcorp said as a result of its dece...
1,"[1, 3267, 699, 3434, 2295, 56, 16784, 7511, 9,...",4,generale de banque sa lt genb br and lt helle...
2,"[1, 53, 12, 284, 15, 14, 272, 26, 53, 959, 32,...",3,shr 3 28 dlrs vs 22 cts shr diluted 2 99 dlrs...
3,"[1, 4, 686, 867, 558, 4, 37, 38, 309, 2276, 46...",4,the farmers home administration the u s agric...
4,"[1, 8295, 111, 8, 25, 166, 40, 638, 10, 436, 2...",4,seton co said its board has received a propos...


In [13]:
X = df["text"].to_numpy()
Y = df["Y"].to_numpy()

In [14]:
X.shape

(11228,)

In [15]:
Y.shape

(11228,)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [17]:
train_tf_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(8)
test_tf_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(8)

In [18]:
classifier = keras_nlp.models.BertClassifier.from_preset(
        "bert_base_en_uncased",
        num_classes=2
    )
   
# Fit the model
classifier.fit(train_tf_dataset, validation_data=test_tf_dataset,  epochs=3)
    

Attaching 'model.safetensors' from model 'keras/bert/keras/bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'model.safetensors.index.json' from model 'keras/bert/keras/bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/bert/keras/bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/bert/keras/bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/bert/keras/bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/bert/keras/bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'model.safetensors' from model 'keras/bert/keras/bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'model.safetensors.index.json' from model 'keras/bert/keras/bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/bert/keras/bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'met

I0000 00:00:1723556702.510442      81 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1723556702.591931      81 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1122/1123 ━━━━━━━━━━━━━━━━━━━━ 0s 641ms/step - loss: nan - sparse_categorical_accuracy: 0.0466

W0000 00:00:1723557521.911897      81 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1123/1123 ━━━━━━━━━━━━━━━━━━━━ 0s 730ms/step - loss: nan - sparse_categorical_accuracy: 0.0466

W0000 00:00:1723557587.543503      83 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1123/1123 ━━━━━━━━━━━━━━━━━━━━ 1004s 788ms/step - loss: nan - sparse_categorical_accuracy: 0.0466 - val_loss: nan - val_sparse_categorical_accuracy: 0.0534
Epoch 2/3
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 781s 695ms/step - loss: nan - sparse_categorical_accuracy: 0.0466 - val_loss: nan - val_sparse_categorical_accuracy: 0.0534
Epoch 3/3
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 781s 695ms/step - loss: nan - sparse_categorical_accuracy: 0.0466 - val_loss: nan - val_sparse_categorical_accuracy: 0.0534


In [19]:
classifier.predict(["What an amazing movie!", "A total waste of my time."])

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


W0000 00:00:1723559156.231209      81 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


array([[nan, nan],
       [nan, nan]], dtype=float32)